In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Descriptors3D, AllChem
import numpy as np

In [2]:
def calculate_descriptors(smiles_list):
    results = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            # Calculate 2D descriptors
            descriptor_dict = {
                'SMILES': smiles,
                'MolWt': Descriptors.MolWt(mol),
                'LogP': Descriptors.MolLogP(mol),
                'NumHAcceptors': Descriptors.NumHAcceptors(mol),
                'NumHDonors': Descriptors.NumHDonors(mol),
                'NumHeteroatoms': Descriptors.NumHeteroatoms(mol),
                'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
                'NumAliphaticCarbocycles': Descriptors.NumAliphaticCarbocycles(mol),
                'NumAliphaticHeterocycles': Descriptors.NumAliphaticHeterocycles(mol),
                'NumAliphaticRings': Descriptors.NumAliphaticRings(mol),
                'NumAromaticCarbocycles': Descriptors.NumAromaticCarbocycles(mol),
                'NumAromaticHeterocycles': Descriptors.NumAromaticHeterocycles(mol),
                'NumAromaticRings': Descriptors.NumAromaticRings(mol),
                'RingCount': Descriptors.RingCount(mol),
                'NHOHCount': Descriptors.NHOHCount(mol),
                'FractionCSP3': Descriptors.FractionCSP3(mol),
                'MinPartialCharge': Descriptors.MinPartialCharge(mol),
                'MaxPartialCharge': Descriptors.MaxPartialCharge(mol),
                'TPSA': Descriptors.TPSA(mol),
            }

            # Calculate 3D descriptors
            mol_3d = Chem.AddHs(mol)
            Chem.AllChem.EmbedMolecule(mol_3d, randomSeed=42)
            if mol_3d.GetNumConformers() > 0:
                descriptor_dict.update({
                    'NPR1': Descriptors3D.NPR1(mol_3d),
                    'NPR2': Descriptors3D.NPR2(mol_3d),
                    'InertialShapeFactor': Descriptors3D.InertialShapeFactor(mol_3d),
                    'RadiusOfGyration': Descriptors3D.RadiusOfGyration(mol_3d),
                    'SpherocityIndex': Descriptors3D.SpherocityIndex(mol_3d),
                })
            else:
                print(f"Warning: 3D conformation generation failed for {smiles}")
                descriptor_dict.update({
                    'NPR1': None,
                    'NPR2': None,
                    'InertialShapeFactor': None,
                    'RadiusOfGyration': None,
                    'SpherocityIndex': None,
                })

            # Calculate ECFP
            ecfp = generate_ecfp(smiles)
            descriptor_dict['ECFP'] = ecfp

            results.append(descriptor_dict)
        else:
            print(f"Could not parse SMILES: {smiles}")

    return results

def generate_ecfp(smiles, radius=3, nBits=2048, use_features=False, use_chirality=True):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return np.zeros(nBits)
    feature_list = AllChem.GetMorganFingerprintAsBitVect(
        molecule,
        radius=radius,
        nBits=nBits,
        useFeatures=use_features,
        useChirality=use_chirality,
    )
    return np.array(feature_list)

In [3]:
# Dictionary of chemical names to trouble shoot and verify the function of the functions above
chemical_names_SMILES_dict = {
    "Ethanol": 'CCO',
    "Acetaminophen": 'CC(=O)Nc1ccc(O)cc1',
    "Sodium bicarbonate": 'O=C([O-])O.[Na+]',
    "Aspirin": 'CC(=O)Oc1ccccc1C(=O)O',
    "Methanol": 'CO',
    "Isopropanol": 'CC(C)O',
    "Glucose": 'OCC1OC(O)C(O)C(O)C1O',
    "Sodium chloride": '[Cl-].[Na+]',
    "Acetic acid": 'CC(=O)O',
}

In [4]:
chemical_SMILES_list = chemical_names_SMILES_dict.values()

the_descriptors = calculate_descriptors(chemical_SMILES_list)

# Print results (you can modify this part as needed)
for result in the_descriptors[:3]:
    print(f"SMILES: {result['SMILES']}")
    print(f"Molecular Weight: {result['MolWt']:.2f}")
    print(f"LogP: {result['LogP']:.2f}")
    print(f"TPSA: {result['TPSA']:.2f}")
    print(f"ECFP (first 50 bits): {result['ECFP'][:50]}")
    print("---")

SMILES: CCO
Molecular Weight: 46.07
LogP: -0.00
TPSA: 20.23
ECFP (first 50 bits): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
---
SMILES: CC(=O)Nc1ccc(O)cc1
Molecular Weight: 151.16
LogP: 1.35
TPSA: 49.33
ECFP (first 50 bits): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
---
SMILES: O=C([O-])O.[Na+]
Molecular Weight: 84.01
LogP: -4.11
TPSA: 60.36
ECFP (first 50 bits): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
---
